In [4]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from PIL import Image, ImageOps
import os, sys
print("TensorFlow version:", tf.__version__)

TensorFlow version: 1.7.0


In [18]:
#rescale and pad with white (3 minutes for 10k 256x256)
# combined https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/ + https://gist.github.com/ihercowitz/642650/f01986c0b1ebd04be588b196eb3ffefe9853e113

def resizeImage(infile, output_dir="scaled", desired_size = 256):
     outfile = os.path.splitext(infile)[0]
     extension = os.path.splitext(infile)[1]

     #if (cmp(extension, ".jpg")):
     if extension != ".jpg":
        return

     if infile != outfile:
        im = Image.open(dir+infile)
        old_size = im.size  # old_size[0] is in (width, height) format
        ratio = float(desired_size)/max(old_size)
        new_size = tuple([int(x*ratio) for x in old_size])
        im = im.resize(new_size, Image.ANTIALIAS)
        new_im = Image.new("RGB", (desired_size, desired_size),color=(255,255,255))
        new_im.paste(im, ((desired_size-new_size[0])//2,(desired_size-new_size[1])//2))
        new_im.save(output_dir+outfile+extension,"JPEG")
        #except IOError:
            #print("cannot reduce image for ", infile)


if __name__=="__main__":
    dir = 'validation/'
    desired_size = 256
    output_dir = "validation256/"
    #dir = os.getcwd()
    

    if not os.path.exists(os.path.join(dir,output_dir)):
        os.mkdir(output_dir)

    for file in os.listdir(dir):
        resizeImage(file,output_dir)

In [5]:
# convert labels
# https://www.kaggle.com/anqitu/for-starter-json-to-multilabel-in-24-seconds
import time
script_start_time = time.time()

import pandas as pd
import numpy as np
import json

pd.set_option('display.max_rows', 600)
pd.set_option('display.max_columns', 50)
import warnings
warnings.filterwarnings('ignore')

data_path = os.getcwd()#"../input"


In [6]:
# 1. Load data =================================================================
print('%0.2f min: Start loading data'%((time.time() - script_start_time)/60))

train={}
test={}
validation={}
with open('%s/train.json'%(data_path)) as json_data:
    train= json.load(json_data)
with open('%s/test.json'%(data_path)) as json_data:
    test= json.load(json_data)
with open('%s/validation.json'%(data_path)) as json_data:
    validation = json.load(json_data)

print('Train No. of images: %d'%(len(train['images'])))
print('Test No. of images: %d'%(len(test['images'])))
print('Validation No. of images: %d'%(len(validation['images'])))

# JSON TO PANDAS DATAFRAME
# train data
train_img_url=train['images']
train_img_url=pd.DataFrame(train_img_url)
train_ann=train['annotations']
train_ann=pd.DataFrame(train_ann)
train=pd.merge(train_img_url, train_ann, on='imageId', how='inner')

# test data
test=pd.DataFrame(test['images'])

# Validation Data
val_img_url=validation['images']
val_img_url=pd.DataFrame(val_img_url)
val_ann=validation['annotations']
val_ann=pd.DataFrame(val_ann)
validation=pd.merge(val_img_url, val_ann, on='imageId', how='inner')

datas = {'Train': train, 'Test': test, 'Validation': validation}
for data in datas.values():
    data['imageId'] = data['imageId'].astype(np.uint32)

print('%0.2f min: Finish loading data'%((time.time() - script_start_time)/60))
print('='*50)

0.00 min: Start loading data
Train No. of images: 1014544
Test No. of images: 39706
Validation No. of images: 9897
0.19 min: Finish loading data


In [7]:
train.head()

,imageId,url,labelId
0,1,https://contestimg.wish.com/api/webimage/570f3...,"[95, 66, 137, 70, 20]"
1,2,https://contestimg.wish.com/api/webimage/5468f...,"[36, 66, 44, 214, 105, 133]"
2,3,https://contestimg.wish.com/api/webimage/54641...,"[170, 66, 97, 153, 105, 138]"
3,4,https://contestimg.wish.com/api/webimage/550b9...,"[18, 66, 44, 153, 164]"
4,5,https://contestimg.wish.com/api/webimage/54451...,"[189, 66, 176, 137, 153, 74, 181, 171, 138]"


In [8]:
trainlabels = train[:].drop(columns=['url'])
trainlabels.head()

,imageId,labelId
0,1,"[95, 66, 137, 70, 20]"
1,2,"[36, 66, 44, 214, 105, 133]"
2,3,"[170, 66, 97, 153, 105, 138]"
3,4,"[18, 66, 44, 153, 164]"
4,5,"[189, 66, 176, 137, 153, 74, 181, 171, 138]"


In [9]:
type(trainlabels['labelId'][0])

list

In [10]:
print([l+'\n' for l in row["labelId"]])

NameError: name 'row' is not defined

In [11]:
#save labels - 4 sec for 10k files

start = time.time()

labeldir = 'image_labels_dir/'
for index, row in trainlabels[:10000].iterrows():
    with open(labeldir+str(row["imageId"])+'.jpg.txt', 'w') as f:
        [f.write("%s\n" % item) for item in row["labelId"]]
        
end = time.time()
print(end - start)    

3.499237060546875


In [ ]:
with open('labels.txt', 'w') as f:
    [f.write("%s\n" % item) for item in row["labelId"]]

In [22]:
s = set()
for item in trainlabels[:10000]["labelId"]:
    u = np.unique(item) 

TypeError: unhashable type: 'numpy.ndarray'

In [35]:
# unique labels list (7 missing in 10k)
labels = []
for item in trainlabels[:10000]["labelId"]:
    for i in item:
        v = int(i)
        if v not in labels:
            labels.append(v)
        else:
            pass

print(len(labels))
labels.sort()
labels

221


[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 157,
 158,
 159,
 160,
 161,
 162,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 

In [34]:
with open('labels.txt', 'w') as f:
    [f.write("%s\n" % item) for item in labels]